In [ ]:
from main import main
import pandas as pd
import numpy as np
import argparse
import shlex

from evaluation import label_evaluation
from utils.utils import mkdir
import json

from sklearn.preprocessing import StandardScaler
from preprocessing import complete_timestamp

In [ ]:
# test_label = np.load('dataset/SMD/SMD_test_label.npy')
# test = np.load('dataset/SMD/SMD_test.npy')
# train = np.load('dataset/SMD/SMD_train.npy')

In [ ]:
def evaluate(preds, labels, kpi_id, missing, timestamp):

    truth = pd.DataFrame({'missing': missing, 'label': labels, 'KPI ID': kpi_id, 'timestamp': timestamp})
    truth[truth['missing'] == 0].to_hdf(truth_file, key='df')

    prediction = pd.DataFrame({'missing': missing, 'predict': preds, 'KPI ID': kpi_id, 'timestamp': timestamp})
    prediction[prediction['missing'] == 0].to_csv(result_file)

    return json.loads(label_evaluation(truth_file, result_file, delay))['data']

In [ ]:
WIN_SIZE = 50

ARG_STRING_TRAIN = '--anormly_ratio 2.1 --num_epochs 1 --batch_size 32 --mode train --dataset ALL-50 --data_path dataset/ALL-50 --input_c 1'
ARG_STRING_TEST = '--anormly_ratio 2.1 --num_epochs 1 --batch_size 32 --mode test --dataset ALL-50 --data_path dataset/ALL-50 --input_c 1 --pretrained_model 20'

In [ ]:
parser = argparse.ArgumentParser()

parser.add_argument('--lr', type=float, default=1e-4)
parser.add_argument('--num_epochs', type=int, default=10)
parser.add_argument('--k', type=int, default=3)
parser.add_argument('--win_size', type=int, default=WIN_SIZE)
parser.add_argument('--input_c', type=int, default=1)
parser.add_argument('--output_c', type=int, default=1)
parser.add_argument('--batch_size', type=int, default=1024)
parser.add_argument('--pretrained_model', type=str, default=None)
parser.add_argument('--dataset', type=str, default='credit')
parser.add_argument('--mode', type=str, default='train', choices=['train', 'test'])
parser.add_argument('--data_path', type=str, default='./dataset/creditcard_ts.csv')
parser.add_argument('--model_save_path', type=str, default='checkpoints')
parser.add_argument('--anormly_ratio', type=float, default=4.00)

config_train = parser.parse_args(shlex.split(ARG_STRING_TRAIN))
config_test = parser.parse_args(shlex.split(ARG_STRING_TEST))

solver_train = main(config_train)
solver_test = main(config_test)

In [ ]:
train_file = '../train.csv'
test_file = '../test.csv'

result_file = 'predict.csv'
truth_file = 'ground_truth.hdf'
delay = 7

train_df = pd.read_csv(train_file).rename(columns={'KPI ID': 'kpi_id'})
test_df = pd.read_csv(test_file).rename(columns={'KPI ID': 'kpi_id'})

In [ ]:
predictions = pd.DataFrame()
for kpi_id in train_df.kpi_id.unique():
    kpi = train_df[train_df['kpi_id'] == kpi_id]
    t_kpi = test_df[test_df['kpi_id'] == kpi_id]
    t_kpi['label'] = 0

    train_timestamp, train_missing, (train_values, train_labels) = complete_timestamp(kpi['timestamp'], (kpi['value'], kpi['label']))
    test_timestamp, test_missing, (test_values, test_labels) = complete_timestamp(t_kpi['timestamp'], (t_kpi['value'], t_kpi['label']))

    window_miss_train = ((len(train_timestamp) // WIN_SIZE + 1) * WIN_SIZE) - len(train_timestamp)

    last_timestamp = train_timestamp[-1]
    final_timestamp = last_timestamp + (60*window_miss_train)

    train_timestamp = np.append(train_timestamp, np.arange(last_timestamp+60, final_timestamp+1, 60))
    train_missing = np.append(train_missing, [1]*window_miss_train)
    train_values = np.append(train_values, [0.]*window_miss_train)
    train_labels = np.append(train_labels, [0]*window_miss_train)

    scaler = StandardScaler()
    scaler.fit(train_values.reshape(-1, 1))

    window_miss_test = ((len(test_timestamp) // WIN_SIZE + 1) * WIN_SIZE) - len(test_timestamp)

    last_timestamp = test_timestamp[-1]
    final_timestamp = last_timestamp + (60*window_miss_test)

    test_timestamp = np.append(test_timestamp, np.arange(last_timestamp+60, final_timestamp+1, 60))
    test_missing = np.append(test_missing, [1]*window_miss_test)
    test_values = np.append(test_values, [0.]*window_miss_test)
    test_labels = np.append(test_labels, [0]*window_miss_test)

    df_test = pd.DataFrame({'timestamp': test_timestamp, 'missing': test_missing, 'value': scaler.transform(test_values.reshape(-1, 1))[:, 0], 'label': test_labels})

    np.save(f'dataset/ALL-50/ALL-50_test.npy', df_test['value'].to_numpy().reshape(-1,1))
    np.save(f'dataset/ALL-50/ALL-50_test_label.npy', df_test['label'].to_numpy())

    solver_test = main(config_test)

    kpi_pred = pd.DataFrame({'missing': test_missing, 'predict': solver_test.pred, 'KPI ID': kpi_id, 'timestamp': test_timestamp})
    predictions = predictions.append(kpi_pred[kpi_pred['missing'] == 0].drop(['missing'], axis=1)).reset_index(drop=True)

___

In [ ]:
predictions.to_csv('predict.csv')

In [ ]:
train_df = pd.read_csv(train_file).rename(columns={'KPI ID': 'kpi_id'})
test_df = pd.read_csv(test_file).rename(columns={'KPI ID': 'kpi_id'})

In [ ]:
all_df = pd.DataFrame()
for kpi_id in train_df.kpi_id.unique():
    kpi = train_df[train_df['kpi_id'] == kpi_id]
    t_kpi = test_df[test_df['kpi_id'] == kpi_id]
    t_kpi['label'] = 0

    train_timestamp, train_missing, (train_values, train_labels) = complete_timestamp(kpi['timestamp'], (kpi['value'], kpi['label']))
    test_timestamp, test_missing, (test_values, test_labels) = complete_timestamp(t_kpi['timestamp'], (t_kpi['value'], t_kpi['label']))

    window_miss_train = ((len(train_timestamp) // WIN_SIZE + 1) * WIN_SIZE) - len(train_timestamp)

    last_timestamp = train_timestamp[-1]
    final_timestamp = last_timestamp + (60*window_miss_train)

    train_timestamp = np.append(train_timestamp, np.arange(last_timestamp+60, final_timestamp+1, 60))
    train_missing = np.append(train_missing, [1]*window_miss_train)
    train_values = np.append(train_values, [0.]*window_miss_train)
    train_labels = np.append(train_labels, [0]*window_miss_train)

    scaler = StandardScaler()
    scaler.fit(train_values.reshape(-1, 1))

    window_miss_test = ((len(test_timestamp) // WIN_SIZE + 1) * WIN_SIZE) - len(test_timestamp)

    last_timestamp = test_timestamp[-1]
    final_timestamp = last_timestamp + (60*window_miss_test)

    test_timestamp = np.append(test_timestamp, np.arange(last_timestamp+60, final_timestamp+1, 60))
    test_missing = np.append(test_missing, [1]*window_miss_test)
    test_values = np.append(test_values, [0.]*window_miss_test)
    test_labels = np.append(test_labels, [0]*window_miss_test)

    df_train = pd.DataFrame({'timestamp': train_timestamp, 'missing': train_missing, 'value': scaler.transform(train_values.reshape(-1, 1))[:, 0], 'label': train_labels})
    df_test = pd.DataFrame({'timestamp': test_timestamp, 'missing': test_missing, 'value': scaler.transform(test_values.reshape(-1, 1))[:, 0], 'label': test_labels})

    all_df = all_df.append(df_train).reset_index(drop=True)

In [ ]:
mkdir(f'dataset/ALL-50')
np.save(f'dataset/ALL-50/ALL-50_train.npy', all_df['value'].to_numpy().reshape(-1,1))
np.save(f'dataset/ALL-50/ALL-50_test.npy', all_df['value'].to_numpy().reshape(-1,1))
np.save(f'dataset/ALL-50/ALL-50_test_label.npy', all_df['label'].to_numpy())